In [1]:
import joblib
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:

# Wczytaj model i encoder
model = joblib.load('recipe_model.pkl')
label_encoder = joblib.load('label_encoder.pkl')

In [3]:
# Składniki do przetestowania
input_ingredients = ['egg', 'milk', 'flour']

# Stwórz pusty DataFrame z wszystkimi możliwymi składnikami
# (kolejność musi być zgodna z tą, którą miało X podczas treningu)
all_ingredients = model.estimators_[0].coef_.shape[1]  # liczba kolumn
mlb = MultiLabelBinarizer()
mlb.fit([input_ingredients])  # tylko po to, żeby mieć strukturę, potem ją nadpiszemy

MultiLabelBinarizer()

In [ ]:
# Możesz zapisać ingredient_df z trenowania i tu go wczytać zamiast tego, ale to uproszczenie:
# Załaduj oryginalny dataset i zrób binarizer dokładnie tak samo jak przy trenowaniu
original_data = pd.read_csv('../data/RecipeNLG_dataset.csv')
original_data.dropna(inplace=True)
original_data['NER'] = original_data['NER'].apply(eval)

MultiLabelBinarizer()

In [ ]:
mlb = MultiLabelBinarizer(sparse_output=True)
mlb.fit(original_data['NER'])  # dopasuj do danych

# Zakoduj składniki
input_vector = mlb.transform([input_ingredients])  # to będzie macierz rzadka
input_df = pd.DataFrame.sparse.from_spmatrix(
    input_vector,
    columns=mlb.classes_
)

In [8]:
# Upewnij się, że kolumny są takie same jak przy trenowaniu
ingredient_columns = model.classes_.shape[0]  # tylko do debugowania

# Uzupełnij brakujące kolumny zerami, jeśli trzeba
for col in model.estimators_[0].feature_names_in_:
    if col not in input_df.columns:
        input_df[col] = 0

In [9]:
# Posortuj kolumny zgodnie z modelem
input_df = input_df[model.estimators_[0].feature_names_in_]

In [ ]:
# Przewidź
predicted_label = model.predict(input_df)[0]
predicted_title = label_encoder.inverse_transform([predicted_label])[0]

In [ ]:
print("Podane składniki:", input_ingredients)
print("Model przewidział przepis:", predicted_title)